#**Neutrino Event Classification with DPCNN Model**

### **Intro: Neutrino Event Classification with DPCNN Model**

Events of Background Alpha and Neutrino Beta
from the *Jinping Simulation Setup* are classified in this notebook.

The [Deep Pyramid Convolution Neural Network](https://www.aclweb.org/anthology/P17-1052.pdf) Method is implemented to generate results approximate to the dataset performance limit.

The model has been optimized and simplified for further research and development. 

Tasks, Datasets and Results could be found [here](https://data-contest.applysquare.com/challenges/ghost-hunter-2020/dataset_files).


### **Intro: Notebook Setups**
The notebook is developed to run online with Google Colab. It also 
supports other notebook setups.

The default work directory is '/content/drive/My Drive'.

In the work directory,
<br>Datasets should be found in './PhysicsData'.
<br>Models will be saved in './Neutrino_DPCNN_Model/'.
<br>Answers will be produced in './DPCNN_Answers/'.

###**System Basics**
Mount Google Drive. 

Check Cuda, GPU, Python and Pytorch.

In [0]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!nvcc -V

In [0]:
# Basics Setup
import sys
print('python version:',sys.version)
import torch
print('torch version:',torch.__version__,end=' | ')
print('cuda available:',torch.cuda.is_available(),end=' | ')

if torch.cuda.is_available():
  print('device count:',torch.cuda.device_count())
  print('current device:',torch.cuda.current_device(),end=' | ')
  print('device name:',torch.cuda.get_device_name(0),end=' | ') # default=current
  print('cuda capability:',torch.cuda.get_device_capability(0)) # default=current 

###**Basic functions for Dataset Processing**

In [0]:
import os
os.chdir('/content/drive/My Drive')

In [0]:
# import os
# os.chdir('/content/drive/My Drive')

import tables
import numpy as np

import math

from torch import nn
import torch

from sklearn.model_selection import train_test_split
import torch.utils.data as Data
from torch.autograd import Variable
from torch import optim

import os

## Basic Functions for Data Processing

# Wave Channel
def make_wave(wave_form,denoise='True'):
    shift = np.argmax(np.bincount(wave_form))  # make baseline shifts, normally 972
    shift_wave = -wave_form + shift  # non_negative_peak + zero_base_level
    cut_wave = shift_wave[5:]
    if denoise != 'False':
      cut_wave[cut_wave<5]=0
    return cut_wave
def make_random_wave(length):
    rand_wave = np.round(np.random.randn(length) / 1.5).tolist()
    return rand_wave

# Event Type
def make_type_vec(type):
    type_vec = [0.5,0.5]
    if type == 0:
        type_vec = [0,1]
    elif type == 1:
        type_vec = [1,0]
    else:
        print('Type Error')
    return type_vec

# Dataset Functions
def make_dataset(WaveTable, TypeTable, StartWaveIndex, StartEventIndex, Period):
    Wave_Index = StartWaveIndex
    Wave_Index_Num = len(WaveTable)
    Event_Index_Num = len(TypeTable)
    Wave_Data = []
    Type_Data = []
    for Particle_Index in range(StartEventIndex, min(StartEventIndex + Period, Event_Index_Num)):
        # Read from TypeTable
        EventID_Particle = TypeTable[Particle_Index]['EventID']
        Type_Particle = TypeTable[Particle_Index]['Alpha']

        # Read WaveTable and make Event Wave
        Event_Wave = []
        for Channel_Index in range(30):
            if Wave_Index < Wave_Index_Num: #Debugged for the Final Wave
                EventID_Waveform = WaveTable[Wave_Index]['EventID']
                ChannelID_Waveform = WaveTable[Wave_Index]['ChannelID']
                Waveform_Waveform = WaveTable[Wave_Index]['Waveform']
                # Process with Abnormal_Check
                if EventID_Waveform < EventID_Particle:
                    print('Abnormal 1, EventScan passes WaveEvent')
                elif EventID_Waveform == EventID_Particle:
                    if ChannelID_Waveform < Channel_Index:
                        print('Abnormal 2, ChannelScan passes WaveChannel')
                    elif ChannelID_Waveform == Channel_Index:
                        Chan_Wave = make_wave(Waveform_Waveform)
                        Wave_Index += 1
                        Event_Wave.append(Chan_Wave)
                    elif ChannelID_Waveform > Channel_Index:
                        Chan_Wave = make_random_wave(1024)
                        Event_Wave.append(Chan_Wave)
                    else:
                        print('Error: Channel Logic Error')
                elif EventID_Waveform > EventID_Particle:
                    Chan_Wave = make_random_wave(1024)
                    Event_Wave.append(Chan_Wave)
                else:
                    print('Error: Event Logic Error')
                if len(Chan_Wave) != 1024:
                    print("length not match")
            else:
              Chan_Wave = make_random_wave(1024)
              Event_Wave.append(Chan_Wave)
        
        # Read Type
        Event_Type = make_type_vec(Type_Particle)

        # Write to Data_Base
        Wave_Data.append(Event_Wave)
        Type_Data.append(Event_Type)

    Wave_Data_NP = np.array(Wave_Data)
    Type_Data_NP = np.array(Type_Data)

    EndWaveIndex = Wave_Index - 1
    EndEventIndex = min(StartEventIndex + Period - 1, Event_Index_Num)
    return Wave_Data_NP, Type_Data_NP, EndWaveIndex, EndEventIndex

print("Basics Functions Defined")

### **DPCNN Structure Defination**

In [0]:
'''Deep Pyramid Convolutional Neural Networks for Text Categorization'''
# Pre-Activation Setup
from torch import nn
import torch.nn.functional as F

class DPCNN(nn.Module):
    def __init__(self, num_filters=250):
        super(DPCNN, self).__init__()
        self.conv_region_embedding = nn.Conv1d(30, num_filters, kernel_size=7, padding=3)
        #Kernel_size 9 is also encouraged, padding should be 4 in these cases, 3/1 is also feasible
        self.conv3 = nn.Conv1d(num_filters, num_filters, kernel_size=3, padding=1)
        self.max_pool = nn.MaxPool1d(kernel_size=3, stride=2)
        self.act_fun = nn.ReLU(True)
        self.bn = nn.BatchNorm1d(num_filters)
        self.fc = nn.Linear(num_filters,2)
        self.channel_size = num_filters
        self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()
    
    def _block(self,x):

        # Pooling
        px = self.max_pool(x)

        # Convolution
        x = self.bn(px)
        x = F.relu(x)
        x = self.conv3(x)

        x = self.bn(px)
        x = F.relu(x)
        x = self.conv3(x)

        # Short Cut
        x = x + px
        return x
    
    def forward(self,x):
        batch = x.shape[0]
        
        # Region embedding
        x = self.conv_region_embedding(x)   
        
        # [batch_size, channel_size, length]
        x = self.act_fun(x)
        x = self.conv3(x)
        x = self.act_fun(x)
        x = self.conv3(x)

        while x.size()[2] > 2:
            x = self._block(x)
                
        x = self.bn(x)
        x = x.view(batch, self.channel_size) #squeeze
        x = self.fc(x)
        x = self.softmax(x)

        return x

print("DPCNN Structure Defined") 

### **Neural Network Initialization**
Start from zero, or reload a pre-trained model

In [0]:
# Start from zero
net = DPCNN()
if torch.cuda.is_available():
    net=net.cuda()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [0]:
# Load a pre-trained model
#net = torch.load("./Neutrino_DPCNN_Model/Model_Name") #Change the Model_Name when needed 
net = torch.load("./Neutrino_DPCNN_Model/Loss_0.4489_R0_pre-0")
if torch.cuda.is_available():
    net=net.cuda()
optimizer = optim.Adam(net.parameters(), lr=5e-6)
#optimizer = optim.SGD(net.parameters(), lr=1e-5, momentum=0.9)

###**Network Training**



In [0]:
## Basic Factors
Division_Period = 5000
BATCHSIZE = 100
EPOCH= 1
ROUND= 20

LoadPath = './PhysicsData/'
FileList = ['pre-0','pre-1','pre-2','pre-3','pre-4']
for round in range(ROUND):
    for FileName in FileList:
        fullfilename = LoadPath + FileName + ".h5"
        # Read hdf5 file
        h5file = tables.open_file(fullfilename, "r")
        WaveformTable = h5file.root.Waveform
        ParticleTruthTable = h5file.root.ParticleTruth
        # Statistics
        print("File", FileName)
        print(len(WaveformTable), "Wave entries")
        print(len(ParticleTruthTable), "Particle entries")

        Wave_Start_Point = 0
        Event_Start_Point = 0
        Training_Division_Num = math.ceil((len(ParticleTruthTable) - Event_Start_Point) / Division_Period)
        # Loop through division parts
        for Division_Index in range(Training_Division_Num):
            #Make Dataset
            Wave_DataSet, Type_DataSet, Wave_End_Point, Event_End_Point = \
                make_dataset(WaveformTable, ParticleTruthTable, Wave_Start_Point, Event_Start_Point, Division_Period)
            print("File", FileName, end=", ")
            print("Event", Event_Start_Point, '-', Event_End_Point, "Created", end=", ")
            print("Wave", Wave_Start_Point, '-', Wave_End_Point, "Processed")

            #Multiple Epochs are supported to increase training efficiency
            for epoch in range(EPOCH):
                #Datasets
                Wave_train, Wave_test, Type_train, Type_test = train_test_split(Wave_DataSet, Type_DataSet, test_size=0.2, random_state=42)
                Wave_train_torch = torch.from_numpy(Wave_train).float()
                Type_train_torch = torch.from_numpy(Type_train).float()
                Wave_test_torch = torch.from_numpy(Wave_test).float()
                Type_test_torch = torch.from_numpy(Type_test).float()
                if torch.cuda.is_available():
                    Wave_train_torch=Wave_train_torch.cuda()
                    Type_train_torch=Type_train_torch.cuda()
                    Wave_test_torch=Wave_test_torch.cuda()
                    Type_test_torch=Type_test_torch.cuda()
                train_data = Data.TensorDataset(Wave_train_torch,Type_train_torch)
                train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCHSIZE, shuffle=True)
                test_data = Data.TensorDataset(Wave_test_torch, Type_test_torch)
                test_loader = Data.DataLoader(dataset=test_data, batch_size=BATCHSIZE, shuffle=False)

                #Training
                running_loss = 0.0
                train_count = 0
                for i, data in enumerate(train_loader, 0):
                    # get the inputs
                    inputs, labels = data
                    inputs, labels = Variable(inputs), Variable(labels)
                    # zero the parameter gradients
                    optimizer.zero_grad()
                    # forward + backward + optimize
                    outputs = net(inputs)
                    criterion = nn.BCELoss()
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.data.item()
                    train_count+=1
                train_loss = running_loss/(train_count)
                print('Training loss: %.5f'%(train_loss),end=", ")

                #Checking
                cum_loss = 0.0
                test_count = 0
                for i, data in enumerate(test_loader, 0):
                    # get the inputs
                    inputs, labels = data
                    inputs, labels = Variable(inputs), Variable(labels)
                    # forward only
                    outputs = net(inputs)
                    criterion = nn.BCELoss()
                    loss = criterion(outputs, labels)
                    cum_loss += loss.data.item()
                    test_count += 1
                test_loss = cum_loss/(test_count)
                print('Testing loss: %.5f'%(test_loss))

            #Division Factor Update
            Wave_Start_Point = Wave_End_Point + 1
            Event_Start_Point = Event_End_Point + 1

        print("h5 file processing finished")
        h5file.close()

        #Save Network Model
        SavePath = './Neutrino_DPCNN_Model/'
        if not os.path.exists(SavePath):
            os.makedirs(SavePath)
        save_name = SavePath+"Loss_"+"%.4f"%(test_loss)+"_R"+str(round)+"_"+FileName
        torch.save(net,save_name)

print("Training Finished")

###**Generating Answers**
Run Directly from trained network, or load from a pre-trained model

In [0]:
# Load a pre-trained model
#net = torch.load("./Neutrino_DPCNN_Model/Model_Name") #Change the Model_Name when needed
net = torch.load("./Neutrino_DPCNN_Model/Loss_0.4489_R0_pre-0")
if torch.cuda.is_available():
    net=net.cuda()

Answer Generation

In [0]:
# import os
# os.chdir('/content/drive/My Drive')

import tables
import numpy as np
import math

from torch import nn
import torch

from sklearn.model_selection import train_test_split
import torch.utils.data as Data
from torch.autograd import Variable
from torch import optim

import os

## Basics Processing Functions
# Wave Channel
def make_wave(wave_form,denoise='True'):
  shift = np.argmax(np.bincount(wave_form))  # make baseline shifts, normally 972
  shift_wave = -wave_form + shift  # non_negative_peak + zero_base_level
  cut_wave = shift_wave[5:]
  if denoise != 'False':
    cut_wave[cut_wave<5]=0
  return cut_wave
def make_random_wave(length):
  rand_wave = np.round(np.random.randn(length) / 1.5).tolist()
  return rand_wave

## Answer Functions
# Calculate Event Num and make ID Count
def make_event_count_and_list(WaveTable):
  ChanNum = len(WaveTable)
  ID_count=0
  ID=[]
  last_ID = -10
  #Search Trough Channel
  for WaveIndex in range(ChanNum):
    eventID = WaveformTable[WaveIndex]['EventID']
    # write in for every new event
    if eventID != last_ID:
      ID_count+=1
      last_ID = eventID
      ID.append(last_ID)
  return ID, ID_count

# Dataset Functions
def make_predict_set(WaveTable, Event_Num, StartWaveIndex, StartEventIndex, Period):
  Wave_Index = StartWaveIndex
  Event_Index_Num = Event_Num
  Wave_Index_Num = len(WaveTable)
  Wave_Data = []
  #Pos_Vec=get_sinusoid_encoding_vec(1024,20)
  for Particle_Index in range(StartEventIndex, min(StartEventIndex + Period, Event_Index_Num+1)):
    # Take Particle_Index
    EventID_Particle = Particle_Index  
    # Read WaveTable and make Event Wave
    Event_Wave = []
    for Channel_Index in range(30):
      if Wave_Index < Wave_Index_Num: #Debugged for the Final Wave
        EventID_Waveform = WaveTable[Wave_Index]['EventID']
        ChannelID_Waveform = WaveTable[Wave_Index]['ChannelID']
        Waveform_Waveform = WaveTable[Wave_Index]['Waveform']
        # Process with Abnormal_Check
        if EventID_Waveform < EventID_Particle:
          print('Abnormal 1, EventScan passes WaveEvent')
          print(EventID_Waveform,EventID_Particle)
          raise SyntaxError('Error: EventScan passes WaveEvent')
        elif EventID_Waveform == EventID_Particle:
          if ChannelID_Waveform < Channel_Index:
            print('Abnormal 2, ChannelScan passes WaveChannel')
          elif ChannelID_Waveform == Channel_Index:
            Chan_Wave = make_wave(Waveform_Waveform)
            Wave_Index += 1
            Event_Wave.append(Chan_Wave)
          elif ChannelID_Waveform > Channel_Index:
            Chan_Wave = make_random_wave(1024)
            Event_Wave.append(Chan_Wave)
          else:
            print('Error: Channel Logic Error')
        elif EventID_Waveform > EventID_Particle:
          Chan_Wave = make_random_wave(1024)
          Event_Wave.append(Chan_Wave)
        else:
          print('Error: Event Logic Error')
        if len(Chan_Wave) != 1024:
          print("length not match")
      else:
        Chan_Wave = make_random_wave(1024)
        Event_Wave.append(Chan_Wave)

    #Event_Wave.extend(Pos_Vec)
    # Write to Data_Base
    Wave_Data.append(Event_Wave)

  Wave_Data_NP = np.array(Wave_Data)
  EndWaveIndex = Wave_Index - 1
  EndEventIndex = min(StartEventIndex + Period - 1, Event_Index_Num)
  return Wave_Data_NP, EndWaveIndex, EndEventIndex
  
################
#net()

## Basic Factors
Division_Period = 4000
BATCHSIZE = 64

LoadPath = "./PhysicsData/"
FileName= "pre-problem"
fullfilename = LoadPath + FileName + ".h5"
# Read hdf5 file
h5file = tables.open_file(fullfilename, "r")
WaveformTable = h5file.root.Waveform
EventList,ID_Num = make_event_count_and_list(WaveformTable) #0 not exist
EventNum = WaveformTable[-1]['EventID']-WaveformTable[0]['EventID']+1

# Statistics
print("File", FileName)
print(len(WaveformTable), "Wave Entries")
print(EventNum, "Particle Entries")
print(ID_Num, "Real Entries")

Wave_Start_Point = 0
Event_Start_Point = 1
Predicting_Division_Num = math.ceil((EventNum - Event_Start_Point) / Division_Period)

# Loop through division parts
Output_Data = []
for Division_Index in range(Predicting_Division_Num):
  #Make Dataset
  Wave_DataSet, Wave_End_Point, Event_End_Point = \
      make_predict_set(WaveformTable, EventNum, Wave_Start_Point, Event_Start_Point, Division_Period)
  print("Event", Event_Start_Point, '-', Event_End_Point, "Created", end=", ")
  print("Wave", Wave_Start_Point, '-', Wave_End_Point, "Processed")

  # Making Dataset
  predict_data = torch.from_numpy(Wave_DataSet).float()
  if torch.cuda.is_available():
    predict_data=predict_data.cuda()
  predict_loader = Data.DataLoader(dataset=predict_data, batch_size=BATCHSIZE, shuffle=False)

  # Makeing Output
  for i, data in enumerate(predict_loader, 0):
    inputs = Variable(data)
    outputs = net(inputs)
    if torch.cuda.is_available():
      batch_output = outputs.cpu().data.numpy()
    else:
      batch_output = outputs.data.numpy()
    batch_predict = batch_output
    Output_Data.extend(batch_predict)

  #Division Factor Update
  Wave_Start_Point = Wave_End_Point + 1
  Event_Start_Point = Event_End_Point + 1

OutputData = np.array(Output_Data)
#Value check inplemented in case of needed
OutputData[OutputData<0]=0
OutputData[OutputData>1]=1
print("Answer ", len(OutputData)," Channels Generated")

################################################

# Make Output Directory
SavePath = "./DPCNN_Answers/"
if not os.path.exists(SavePath):
    os.makedirs(SavePath)

# Writing a file.
# Define the database columns
class AnswerData(tables.IsDescription):
    EventID = tables.Int64Col(pos=0)
    Alpha = tables.Float32Col(pos=1)

# Create the output file and the group
h5file_write = tables.open_file(SavePath + "DPCNN_Neutrino_Predict.h5", mode="w", title="OneTonDetector")
# Create tables
AnswerTable = h5file_write.create_table("/", "Answer", AnswerData, "Answer")
answer = AnswerTable.row

for j in EventList:
    answer['EventID'] = j
    answer['Alpha'] = OutputData[j-1][0]
    answer.append()
# Flush into the output file

AnswerTable.flush()
print(len(AnswerTable), " Events")

h5file_write.close()
h5file.close()

print("Answer Produced")


###**Other Functions**

In [0]:
# check void wave record event(s)
jump=0
for i in range(len(EventList)):
  if EventList[i] != i+jump:
    print("event do not exist",i+jump)
    jump+=1

print(jump,' non-input event(s)')
